Add logistic regression score from all features.

In [15]:
import json
from sklearn.linear_model import LogisticRegression

In [16]:
path_train = 'test-B-big/dev.jl'

In [17]:
def read_rows(path):
    rows = []
    for line in open(path):
        rows.append(json.loads(line))
    return rows

def write_json_format(path_out, rows):
    f_out = open(path_out, 'w')
    for row in rows:
        f_out.write(json.dumps(row, ensure_ascii=False)+'\n')

rows_train = read_rows(path_train)

In [18]:
X_train = [[
    row['score_bm25'], 
    row['score_bm25_not_lemmatized'], 
    row['score_bm25_bigrams'], 
    row['score_miniLM'],
    row['score_miniLM_with_gpt3'],
    row['score_miniLM_with_chatgpt'],
    row['score_miniLM_gpt3_answer'],
    row['score_miniLM_chatgpt_answer'],
] for row in rows_train]
y_train = [row['label'] for row in rows_train]

In [19]:
model = LogisticRegression().fit(X_train, y_train)

In [20]:
paths_test = ['test-B-big/wiki.jl', 'test-B-big/legal.jl', 'test-B-big/allegro.jl']

In [21]:
for path_test in paths_test:
    rows_test = read_rows(path_test)
    X_test = [[
        row['score_bm25'], 
        row['score_bm25_not_lemmatized'], 
        row['score_bm25_bigrams'], 
        row['score_miniLM'],
        row['score_miniLM_with_gpt3'],
        row['score_miniLM_with_chatgpt'],
        row['score_miniLM_gpt3_answer'],
        row['score_miniLM_chatgpt_answer'],
    ] for row in rows_test]
    probs = model.predict_proba(X_test)
    for row, prob in zip(rows_test, probs):
        row['log_reg_full_with_chatgpt'] = prob[1]
    write_json_format(path_test, rows_test)